In [8]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [16]:
import numpy as np
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

# Your code here

# Reset warnings filter to default
warnings.filterwarnings('default')

# Mount Google Drive


# Change directory to where the npz file is located
%cd /content/drive/MyDrive/

# Load npz file
data = np.load('g_srgan (1).npz')



/content/drive/MyDrive


In [17]:
data

In [18]:
data = np.load('g_srgan (1).npz',allow_pickle=True)

# Print keys
print(data.keys())



KeysView(<numpy.lib.npyio.NpzFile object at 0x7fb4d304da80>)


In [19]:
 #Convert to dictionary-like object
data_dict = dict(data.items())

# Print contents of the dictionary
print(data_dict)

{'params': array([array([[[[-0.01663417, -0.02019958,  0.0217146 , ..., -0.05426534,
                 -0.00884723,  0.00366171],
                [ 0.04544824,  0.03112731, -0.01977636, ...,  0.07980285,
                  0.03575801,  0.03467239],
                [-0.0197451 , -0.02987451,  0.03479709, ..., -0.01887666,
                 -0.05819398,  0.06451631]],

               [[ 0.01511194, -0.05284986,  0.03910235, ..., -0.072881  ,
                 -0.01080778,  0.0290093 ],
                [ 0.09855222,  0.0571218 , -0.05620818, ...,  0.05744448,
                 -0.01912365,  0.0553307 ],
                [-0.09522071, -0.04137253,  0.01614104, ...,  0.04167936,
                 -0.03971417,  0.05649997]],

               [[-0.00291743, -0.01024524,  0.0648189 , ...,  0.00744869,
                 -0.03294221,  0.00814333],
                [ 0.04645391,  0.04095214, -0.02192828, ..., -0.1021098 ,
                 -0.00189963,  0.03494181],
                [-0.07093937, -0.01327882

In [20]:




# Access variables
if 'x' in data.keys():
    x = data['x']
else:
    print("The key 'x' does not exist in the npz file.")
    
if 'y' in data.keys():
    y = data['y']
else:
    print("The key 'y' does not exist in the npz file.")


The key 'x' does not exist in the npz file.
The key 'y' does not exist in the npz file.


In [21]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Load the pre-trained model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name,output_hidden_states=True)

# Define a function to generate MLM predictions
def generate_predictions(text):
    # Tokenize the input text
    input_ids = tokenizer.encode(text, return_tensors='pt')

    # Generate MLM predictions
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs[0][0]

    # Convert predictions to probabilities
    probabilities = torch.softmax(predictions, dim=-1)

    # Convert probabilities to numpy array
    numpy_probs = probabilities.numpy()

    # Return the predicted tokens and their probabilities
    predicted_tokens = tokenizer.convert_ids_to_tokens(torch.argmax(predictions, dim=-1))
    predicted_probabilities = numpy_probs[range(len(predicted_tokens)), torch.argmax(predictions, dim=-1)]

    return predicted_tokens, predicted_probabilities



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
# Example usage
text = "The quick brown [MASK] jumps over the lazy dog"
predicted_tokens, predicted_probabilities = generate_predictions(text)
print(predicted_tokens)
print(predicted_probabilities)


['.', 'the', 'quick', 'brown', 'dog', 'jumps', 'over', 'the', 'lazy', '.', '.']
[0.02983472 0.9975522  0.46510053 0.9727898  0.13713929 0.7987209
 0.99981064 0.99996746 0.9930011  0.97335106 0.98052406]


['.', 'john', 'works', 'in', '[', 'apple', ']', '.']
[0.0264926  0.47895017 0.9996061  0.99764496 0.9999821  0.9999676
 0.8052849  0.99980885]
